In [1]:
from transformers import GPT2LMHeadModel

/home/mikolaj/.cache/pypoetry/virtualenvs/gpt2-from-scratch-j40tdloB-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import tiktoken

In [3]:
import torch
enc = tiktoken.get_encoding("gpt2")

In [4]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"\\gpt2_from_scratch")


In [5]:
from gpt2_from_scratch.GPT import GPT
from gpt2_from_scratch.GPTConfig import GPTConfig

In [6]:
from torch.nn import functional as F

model = GPT(GPTConfig(vocab_size=50304))
checkpoint = torch.load("/home/mikolaj/work/GPT2_from_scratch/gpt2_from_scratch/log/model_00100.pt", map_location="cpu")
state_dict = checkpoint['model_state_dict']
model.load_state_dict(state_dict)

device = "cuda"
model.to(device)

max_length = 20
num_return_seq = 5

tokens = enc.encode("Hello, I am")
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_seq, 1)
xgen = tokens.to(device)
sample_rng = torch.Generator(device=device)
sample_rng.manual_seed(42)

while xgen.size(1) < max_length:
    with torch.no_grad():
        logits, loss = model(xgen)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng)
        xcol = torch.gather(topk_indices, -1, ix)
        xgen = torch.cat((xgen, xcol), dim=1)
for i in range(num_return_seq):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(f"sample {i}: {decoded}")


/tmp/ipykernel_6574/1205284559.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/home/mikolaj/work/GPT2_from_scratch/gpt2_from_scratch/log/model_

sample 0: Hello, I am at the--The own in "(F (p, are these in
sample 1: Hello, I am� York from the "� be of the "M' it in to�
sample 2: Hello, I am, the work to the use but their� was the the own,, the
sample 3: Hello, I am in.
This the same and their� "The new from the new and
sample 4: Hello, I am the child, for the National/For the�).
H.--
